## **HyperParameter tuning a Neural Network**

### Using **Keras Tuner**

`pip install tensorflow`

`pip install keras-tuner`

Using a Diabates dataset

Dataset link : [Diabates dataset](https://www.kaggle.com/datasets/saurabh00007/diabetescsv)

In [ ]:
# importing necessary libraries

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow
from tensorflow import keras
from keras import Sequential, layers

import keras_tuner as kt

In [69]:
df = pd.read_csv('../datasets/diabetes.csv')
print(f"Shape of the dataset: {df.shape}")
df.head()

Shape of the dataset: (768, 9)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [70]:
# correlation

df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [71]:
# scaling

X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values


sc = StandardScaler()
X = sc.fit_transform(X)

In [ ]:
# train test splitting

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [73]:
print(f"X shape: {X.shape}")
print(f"Train size: {x_train.shape}")
print(f"Test shape: {x_test.shape}")

X shape: (768, 8)
Train size: (614, 8)
Test shape: (154, 8)


In [74]:
# building a simple binary classification model

model = Sequential([
    layers.Input(shape=(8,)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss = keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.BinaryAccuracy(name='accuracy')]
)

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# training 

history = model.fit(
    x_train, y_train,
    batch_size=32,
    epochs=10,
    validation_split=0.2 # this will cosider some part of train data as validation data
)

Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.4644 - loss: 0.7139 - val_accuracy: 0.6179 - val_loss: 0.6713
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6171 - loss: 0.6535 - val_accuracy: 0.6911 - val_loss: 0.6328
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6864 - loss: 0.6101 - val_accuracy: 0.6829 - val_loss: 0.6040
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7352 - loss: 0.5770 - val_accuracy: 0.6748 - val_loss: 0.5812
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7454 - loss: 0.5528 - val_accuracy: 0.6829 - val_loss: 0.5632
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7475 - loss: 0.5340 - val_accuracy: 0.6829 - val_loss: 0.5487
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7536 - loss: 0.5190 - val_accuracy: 0.7236 - val_loss: 0.5367
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7658 - loss: 0.5061 - val_accuracy: 0.7317 - val_loss

We will perform
* How to select appropriate optimizer
* Number of nodes in a layer
* Selecting number of layers
* All the things in one model

#### **selecting appropriate optimizer**

In [ ]:
def build_model(hp):
    model = Sequential([
        layers.Input(shape=(8,)),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=hp.Choice('optimizer', values = ['adam', 'sgd', 'rmsprop', 'adadelta']),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

In [78]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    directory = 'HyperParamTuning',
    project_name='Optimizers',
    max_trials=5 # tuner selects 5 sets of hyperparameters and model for each combination set
)

In [79]:
tuner.search(
    x_train,
    y_train,
    epochs=5,
    validation_data=(x_test, y_test)
)

Trial 4 Complete [00h 00m 03s]
val_accuracy: 0.6818181872367859

Best val_accuracy So Far: 0.7142857313156128
Total elapsed time: 00h 00m 10s


In [81]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [82]:
model = tuner.get_best_models(num_models=1)[0]

d:\srinivas\trainee-assignment-2025\training_venv\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [83]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# training

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, initial_epoch=6)

Epoch 7/100


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.7443 - loss: 0.5248 - val_accuracy: 0.6883 - val_loss: 0.5484
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7541 - loss: 0.5072 - val_accuracy: 0.7013 - val_loss: 0.5366
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7508 - loss: 0.4955 - val_accuracy: 0.7143 - val_loss: 0.5282
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7541 - loss: 0.4863 - val_accuracy: 0.7338 - val_loss: 0.5224
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7573 - loss: 0.4794 - val_accuracy: 0.7273 - val_loss: 0.5183
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7720 - loss: 0.4734 - val_accuracy: 0.7338 - val_loss: 0.5146
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7736 - loss: 0.4691 - val_accuracy: 0.7338 - val_loss: 0.5117
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7769 - loss: 0.4647 - val_accuracy: 0.7403 - val_los

#### **finding the number of nodes**

In [ ]:
def build_model(hp):
    model = Sequential([
        layers.Input(shape=(8,)),
        layers.Dense(hp.Int('units', min_value=8, max_value=128, step=32), activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(
        optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy']
    )
    return model

In [86]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='HyperParamTuning',
    project_name='num_nodes'
)

Reloading Tuner from HyperParamTuning\num_nodes\tuner0.json


In [87]:
tuner.search(
    x_train,
    y_train,
    validation_data=(x_test, y_test),
    epochs=5
)

In [88]:
tuner.get_best_hyperparameters()[0].values

{'units': 104}

In [89]:
model = tuner.get_best_models(num_models=1)[0]

d:\srinivas\trainee-assignment-2025\training_venv\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


#### **number of layers**

In [ ]:
from keras.layers import Dense

def build_model(hp):
    inputs = keras.Input(shape=(8,))
    x = layers.Dense(72, activation='relu')(inputs)
    
    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
        x = layers.Dense(72, activation='relu')(x)
    
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer = keras.optimizers.RMSprop(),
        loss = keras.losses.BinaryCrossentropy(),
        metrics = [keras.metrics.BinaryAccuracy(name='accuracy')]
    )

    return model


In [100]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=6,
    directory='HyperParamTuning',
    project_name = 'num_layers'
)

In [101]:
tuner.search(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs = 5,
)

Trial 6 Complete [00h 00m 04s]
val_accuracy: 0.7792207598686218

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 21s


In [102]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2}

In [103]:
model = tuner.get_best_models(num_models=1)[0]

d:\srinivas\trainee-assignment-2025\training_venv\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


#### **all the things in one**

In [ ]:
def build_model(hp):
    inputs = keras.Input(shape=(8,))
    x = inputs

    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
        x = layers.Dense(
            units=hp.Int(f"units_{i}", min_value=8, max_value=128, step=32),
            activation=hp.Choice(f"activation_{i}", values = ['relu', 'tanh', 'sigmoid'])
        )(x)

    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs, outputs)

    optimizer_choice = hp.Choice('optimizer', values = ['adam', 'rmsprop', 'adadelta', 'sgd', 'nadam'])
    optimizers = {
        'adam': keras.optimizers.Adam(),
        'rmsprop': keras.optimizers.RMSprop(),
        'adadelta': keras.optimizers.Adadelta(),
        'sgd': keras.optimizers.SGD(),
        'nadam': keras.optimizers.Nadam(),
    }

    model.compile(
        optimizer = optimizers[optimizer_choice],
        loss = keras.losses.BinaryCrossentropy(),
        metrics = [keras.metrics.BinaryAccuracy(name='accuracy')]
    )

    return model

In [106]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    directory='HyperParamTuning',
    project_name='AllHypParams'
)

In [107]:
tuner.search(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs = 5
)

Trial 10 Complete [00h 00m 04s]
val_accuracy: 0.3571428656578064

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 42s


In [108]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 4,
 'units_0': 72,
 'activation_0': 'tanh',
 'optimizer': 'rmsprop',
 'units_1': 40,
 'activation_1': 'relu',
 'units_2': 72,
 'activation_2': 'tanh',
 'units_3': 40,
 'activation_3': 'sigmoid',
 'units_4': 104,
 'activation_4': 'sigmoid',
 'units_5': 8,
 'activation_5': 'tanh',
 'units_6': 8,
 'activation_6': 'tanh',
 'units_7': 104,
 'activation_7': 'sigmoid'}

In [109]:
model = tuner.get_best_models(num_models=1)[0]

d:\srinivas\trainee-assignment-2025\training_venv\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
# training 

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, initial_epoch=6)

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.7687 - loss: 0.4747 - val_accuracy: 0.7792 - val_loss: 0.4985
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7866 - loss: 0.4582 - val_accuracy: 0.7727 - val_loss: 0.4939
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7720 - loss: 0.4522 - val_accuracy: 0.7597 - val_loss: 0.5012
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7915 - loss: 0.4467 - val_accuracy: 0.7597 - val_loss: 0.5035
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7899 - loss: 0.4431 - val_accuracy: 0.7532 - val_loss: 0.5024
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7801 - loss: 0.4395 - val_accuracy: 0.7468 - val_loss: 0.5062
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7834 - loss: 0.4348 - val_accuracy: 0.7468 - val_loss: 0.5111
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7997 - loss: 0.4301 - val_accuracy: 0